# Open AI -mallien hienosäätö

Tämä muistio perustuu Open AI:n [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) -dokumentaatiossa annettuihin ajankohtaisiin ohjeisiin.

Hienosäätö parantaa perustamallien suorituskykyä sovelluksessasi kouluttamalla mallia uudelleen lisädatalla ja kontekstilla, jotka liittyvät juuri kyseiseen käyttötapaukseen tai tilanteeseen. Huomaa, että kehoteinsinöörin tekniikat kuten _few shot learning_ ja _retrieval augmented generation_ mahdollistavat oletuskehotteen parantamisen asiaankuuluvalla datalla laadun parantamiseksi. Näitä lähestymistapoja kuitenkin rajoittaa kohdemallin maksimimerkkimäärä.

Hienosäädössä koulutamme itse mallia uudelleen tarvittavalla datalla (mikä mahdollistaa huomattavasti useampien esimerkkien käytön kuin mitä maksimimerkkimäärä sallisi) ja otamme käyttöön _räätälöidyn_ version mallista, jolle ei enää tarvitse antaa esimerkkejä ennustevaiheessa. Tämä ei ainoastaan paranna kehotteen suunnittelun tehokkuutta (meillä on enemmän joustavuutta käyttää merkkimäärää muihin tarkoituksiin), vaan voi myös pienentää kustannuksia (koska mallille tarvitsee lähettää vähemmän merkkejä ennustevaiheessa).

Hienosäätöön kuuluu 4 vaihetta:
1. Valmistele koulutusdata ja lataa se.
1. Suorita koulutusprosessi ja saat hienosäädetyn mallin.
1. Arvioi hienosäädetty malli ja toista prosessia laadun parantamiseksi.
1. Ota hienosäädetty malli käyttöön ennustamista varten, kun olet tyytyväinen tuloksiin.

Huomaa, että kaikki perustamallit eivät tue hienosäätöä – [tarkista OpenAI:n dokumentaatiosta](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) ajantasaiset tiedot. Voit myös hienosäätää aiemmin hienosäädetyn mallin. Tässä ohjeessa käytämme `gpt-35-turbo` -mallia hienosäädön kohteena. 

---


### Vaihe 1.1: Valmistele datasi

Rakennetaan chatbot, joka auttaa sinua ymmärtämään jaksollista järjestelmää vastaamalla kysymyksiin alkuaineista limerikin muodossa. Tässä yksinkertaisessa ohjeessa luomme vain pienen opetusdatan, jossa on muutama esimerkkivastaus, jotka näyttävät, millaisessa muodossa datan tulisi olla. Oikeassa käyttötapauksessa tarvitsisit paljon enemmän esimerkkejä sisältävän aineiston. Voit myös hyödyntää avointa dataa (sovellusalueeseesi liittyen), jos sellaista on saatavilla, ja muokata sen sopivaksi hienosäätöä varten.

Koska keskitymme `gpt-35-turbo`-malliin ja haluamme yksittäisen vastauksen (chat completion), voimme tehdä esimerkit [tämän suositellun muodon](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) mukaisesti, joka vastaa OpenAI:n chat completion -vaatimuksia. Jos odotat monivaiheista keskustelua, käyttäisit [monivaiheista esimerkkimuotoa](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), jossa on mukana `weight`-parametri kertomassa, mitkä viestit otetaan mukaan (tai jätetään pois) hienosäätöprosessissa.

Tässä ohjeessa käytämme yksinkertaisempaa yksittäisen vuoron muotoa. Data on [jsonl-muodossa](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst), jossa jokainen rivi on yksi tietue JSON-muodossa. Alla olevassa esimerkissä on kaksi tietuetta – katso [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) nähdäksesi koko esimerkkiaineiston (10 esimerkkiä), jota käytämme tässä hienosäätöohjeessa. **Huom:** Jokainen tietue _täytyy_ olla yhdellä rivillä (ei jaettuna usealle riville kuten muotoillussa JSON-tiedostossa yleensä).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Oikeassa käyttötapauksessa tarvitset paljon suuremman esimerkkiaineiston saadaksesi hyviä tuloksia – joudut tasapainottelemaan vastausten laadun ja hienosäädön ajan/kustannusten välillä. Käytämme pientä aineistoa, jotta hienosäätöprosessi on nopea ja voimme havainnollistaa sen kulun. Katso [tämä OpenAI Cookbook -esimerkki](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) monimutkaisempaan hienosäätöohjeeseen.


### Vaihe 1.2 Lataa oma tietoaineistosi

Lataa data Files API:n avulla [kuten täällä kuvataan](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Huomaa, että ennen tämän koodin suorittamista sinun tulee olla tehnyt seuraavat asiat:
 - Asentanut `openai` Python-paketin (varmista, että käytät versiota >=0.28.0 saadaksesi uusimmat ominaisuudet)
 - Asettanut `OPENAI_API_KEY`-ympäristömuuttujan OpenAI API -avaimellesi
Lisätietoja löydät kurssin [Asennusoppaasta](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst).

Suorita nyt koodi, jolla luot tiedoston ladattavaksi paikallisesta JSONL-tiedostostasi.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Vaihe 2.1: Luo hienosäätötehtävä SDK:lla


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Vaihe 2.2: Tarkista työn tila

Tässä on muutamia asioita, joita voit tehdä `client.fine_tuning.jobs` API:lla:
- `client.fine_tuning.jobs.list(limit=<n>)` – Listaa viimeiset n hienosäädön työtä
- `client.fine_tuning.jobs.retrieve(<job_id>)` – Hae tietyn hienosäädön työn tiedot
- `client.fine_tuning.jobs.cancel(<job_id>)` – Peru hienosäädön työ
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` – Listaa enintään n tapahtumaa työstä
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Prosessin ensimmäinen vaihe on _koulutustiedoston validointi_, jotta varmistetaan datan oikea muoto.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Vaihe 2.3: Seuraa tapahtumia edistymisen seuraamiseksi


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Vaihe 2.4: Tarkastele tilaa OpenAI-hallintapaneelissa


Voit myös tarkistaa tilan vierailemalla OpenAI:n verkkosivuilla ja tutkimalla alustan _Fine-tuning_-osiota. Tämä näyttää sinulle nykyisen työn tilan ja antaa mahdollisuuden seurata aiempien suorituskertojen historiaa. Tässä kuvakaappauksessa näet, että aiempi suoritus epäonnistui ja toinen yritys onnistui. Taustatietona: tämä tapahtui, kun ensimmäisessä ajossa käytettiin JSON-tiedostoa, jossa oli väärin muotoiltuja tietueita – kun tämä korjattiin, toinen ajo valmistui onnistuneesti ja malli tuli käyttöön.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.fi.png)


Voit myös tarkastella tilaviestejä ja mittareita vierittämällä alaspäin visuaalisessa hallintapaneelissa, kuten alla näkyy:

| Viestit | Mittarit |
|:---|:---|
| ![Viestit](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.fi.png) |  ![Mittarit](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.fi.png)|


### Vaihe 3.1: Hae ID & testaa hienosäädettyä mallia koodissa


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Vaihe 3.2: Lataa ja testaa hienosäädetty malli Playgroundissa

Voit nyt testata hienosäädettyä mallia kahdella tavalla. Ensinnäkin voit siirtyä Playgroundiin ja valita mallivalikosta juuri hienosäädetyn mallisi. Toinen vaihtoehto on käyttää hienosäädön paneelissa näkyvää "Playground"-vaihtoehtoa (katso yllä oleva kuvakaappaus), joka avaa _vertailunäkymän_, jossa perusmalli ja hienosäädetty malli näkyvät rinnakkain nopeaa arviointia varten.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.fi.png)

Syötä vain koulutusdatassasi käytetty järjestelmäkonteksti ja anna testikysymyksesi. Huomaat, että molemmille puolille päivittyy sama konteksti ja kysymys. Suorita vertailu ja näet, miten mallien vastaukset eroavat toisistaan. _Huomaa, miten hienosäädetty malli tuottaa vastauksen esimerkeissäsi määrittelemässäsi muodossa, kun taas perusmalli seuraa vain järjestelmäkehotetta._

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.fi.png)

Huomaat myös, että vertailunäkymä näyttää kummankin mallin token-määrät sekä inferenssiin kuluneen ajan. **Tämä esimerkki on yksinkertainen ja tarkoitettu vain prosessin havainnollistamiseen, eikä se vastaa todellista dataa tai tilannetta.** Saatat huomata, että molemmissa esimerkeissä token-määrä on sama (koska järjestelmäkonteksti ja käyttäjän kysymys ovat identtiset), mutta hienosäädetty malli käyttää enemmän aikaa inferenssiin (koska kyseessä on mukautettu malli).

Todellisissa käyttötapauksissa et käytä näin yksinkertaista esimerkkiä, vaan hienosäädät mallia oikealla datalla (esim. tuotekatalogi asiakaspalvelussa), jolloin vastausten laatu korostuu selkeämmin. _Tällöin_ perusmallilla vastaavan laatutason saavuttaminen vaatii enemmän räätälöityä kehotteiden suunnittelua, mikä kasvattaa token-määrää ja mahdollisesti myös inferenssiaikaa. _Jos haluat kokeilla tätä, tutustu OpenAI Cookbookin hienosäätöesimerkkeihin aloittaaksesi._



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Pyrimme tarkkuuteen, mutta huomioithan, että automaattiset käännökset saattavat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulee pitää ensisijaisena lähteenä. Kriittisissä tapauksissa suosittelemme ammattimaisen ihmiskääntäjän käyttöä. Emme ole vastuussa tämän käännöksen käytöstä mahdollisesti aiheutuvista väärinkäsityksistä tai tulkinnoista.
